In [ ]:
import os
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.co

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

--2022-03-16 09:14:39--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-03-16 09:14:40 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_da

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

In [ ]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [ ]:
total_votes_df = vine_df.filter(vine_df.total_votes >= 20)
total_votes_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R2QAPWF1M5863L|          1|            5|         23|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R28WZIA84DUP6C|          2|            4|         43|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|     

In [ ]:
percent_votes_df = total_votes_df.filter((total_votes_df.helpful_votes / total_votes_df.total_votes) >= .50)
percent_votes_df.show(10)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|
| RTX63ENBKWCXF|          1|           28|         35|   N|     

In [ ]:
vine_yes_df = percent_votes_df.filter(percent_votes_df.vine == 'Y')
vine_yes_df.show(10)

vine_no_df = percent_votes_df.filter(percent_votes_df.vine == 'N')
vine_no_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R397PIJYYVJ7PD|          4|           31|         32|   Y|                N|
| RP6Y2CTIFUYD0|          2|           31|         32|   Y|                N|
|R1FYRNM5U0S2VZ|          3|           20|         24|   Y|                N|
| R1ME19UY4UNAO|          5|           53|         57|   Y|                N|
|R1JW7ZHT2VLT1E|          3|           40|         45|   Y|                N|
| R98UYXY43TURV|          4|           46|         50|   Y|                N|
|R34VXM3EJDS78Z|          3|          374|        410|   Y|                N|
|R1QX8UOW5JO6L0|          4|           20|         21|   Y|                N|
| R5Y5SXCRQ2GNH|          3|           34|         37|   Y|     

In [ ]:
total_vine_yes = vine_yes_df.count()
vine_yes_5star = vine_yes_df.filter(vine_yes_df.star_rating == 5).count()
vine_yes_5star_percentage = (vine_yes_5star/total_vine_yes) * 100
print(total_vine_yes, vine_yes_5star, vine_yes_5star_percentage)

total_vine_no = vine_no_df.count()
vine_no_5star = vine_no_df.filter(vine_no_df.star_rating == 5).count()
vine_no_5star_percentage = (vine_no_5star/total_vine_no) *100
print(total_vine_no, vine_no_5star, vine_no_5star_percentage)

5012 2031 40.52274541101357
109297 49967 45.716716835777746
